## Merging dataset on key - POP2-Lab-HEMASPOT-IFA-RedCap

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, output_file, show,output_notebook
output_notebook()

from HUGfunctions import calculateAge, create_households, convertDate

forPublic=True

Loading BokehJS ...

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import FuncFormatter, MaxNLocator

Create a timestamp

In [3]:
# current date and time
now = datetime.now()

timestamp = timestampStr = now.strftime("%d-%b-%Y_%Hh%M")
print("timestamp =", timestamp)

timestamp = 25-May-2020_15h25


Paths and names of the relevant files

In [4]:
path='/Users/euclide/SynologyDrive/Mac/HUGWork/'


##inputs
#pop2
pop2=['pop_2_Final_allUpdated_2020-05-20_17h45.xlsx','Sheet1']

#lab
lab=['2020.05.16_lab_update_correct.xls','Sheet1']

#RedCap
redcap=['REDCap_clean_16_05.xls','Sheet1']

##supporting files


#outputs
#FullOutput
outputName='Merge_Pop2_Lab_IFA_Hemaspot_RedCap_'+timestamp+'.xlsx'

# Create the all dataframes

Pop df

In [5]:
df_pop = pd.read_excel(io=path+pop2[0], sheet_name=pop2[1])
if !forPublic : 
    df_pop.head()

SyntaxError: invalid syntax (<ipython-input-5-889ef7016f8e>, line 2)

Lab df (normally lab file includes also IFA)

In [ ]:
df_lab = pd.read_excel(io=path+lab[0], sheet_name=lab[1])
if !forPublic : 
    df_lab.head()

RedCap df

In [ ]:
df_redcap = pd.read_excel(io=path+redcap[0], sheet_name=redcap[1])
if !forPublic : 
    df_redcap.head()

# Data manipulation

Pop2 cleaning

In [ ]:
print("Initial pop2 size         = "+str(df_pop.shape[0]))
df_pop=df_pop[(df_pop['etat2']==10) | (df_pop['etat2']==99) | (df_pop['etat2']==12) | (df_pop['etat2']==19)]
print("After-filtering pop2 size = "+str(df_pop.shape[0]))

Convert date

In [ ]:
df_pop['datededebut']=df_pop['datededebut'].apply(lambda x: convertDate(x))
df_pop['ddn'] = df_pop['ddn'].apply(lambda x: convertDate(x) )
df_pop["codbar"].groupby(df_pop["datededebut"].astype("datetime64").dt.month).count().plot(kind="bar")

Calculate age

In [ ]:
df_pop['age']=df_pop['ddn'].apply(lambda x: calculateAge(x))
df_pop.hist(column='age')

Household codbar

In [ ]:
df_pop['Household_codbar']=df_pop['codbar'].apply(lambda x: create_households(int(x)))

Cleaning of sensitive data

In [ ]:
df_pop=df_pop[['codbar','Household_codbar','sex','ddn','age','datededebut','etat2']]

# Actual merging

In [ ]:
#make sure for all the files the codbar is a string
df_pop['codbar']=df_pop['codbar'].apply(lambda x:  str(x))
df_lab['Codbar']=df_lab['Codbar'].apply(lambda x:  str(x))
df_redcap['Codbar']=df_redcap['Codbar'].apply(lambda x:  str(x))

df_merge=df_pop.merge(df_lab,how='left',left_on='codbar', right_on='Codbar')
df_merge=df_merge.merge(df_redcap,how='left',left_on='codbar', right_on='Codbar')

if !forPublic : 
    df_merge.head()

# Saving the results

In [ ]:
df_merge.to_excel(path+outputName, index = False)